# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Create worspace and experiment

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'ml-capstone-automl'

experiment=Experiment(ws, experiment_name)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
dataset = Dataset.get_by_name(ws, name = 'Diabetes-predict')

In [4]:
dataset.take(5).to_pandas_dataframe()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


## Connect to compute target

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook240351"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diabetes",  
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-capstone-automl,AutoML_cc6d7769-4adb-4035-ae29-b081ed4b1bd6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_cc6d7769-4adb-4035-ae29-b081ed4b1bd6',
 'target': 'notebook240351',
 'status': 'Completed',
 'startTimeUtc': '2023-08-17T06:06:39.395212Z',
 'endTimeUtc': '2023-08-17T06:18:55.85336Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook240351',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-capstone-automl","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-240351","workspace_name":"quick-starts-ws-240351","region":"westus2","compute_target":"notebook2403

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Retrieve best model from Pipeline Run
best_automl_run, fitted_model = remote_run.get_output()
print(best_automl_run.get_details())
print(best_automl_run.get_metrics())

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

{'runId': 'AutoML_cc6d7769-4adb-4035-ae29-b081ed4b1bd6_21', 'target': 'notebook240351', 'status': 'Completed', 'startTimeUtc': '2023-08-17T06:12:22.732505Z', 'endTimeUtc': '2023-08-17T06:12:58.499635Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': 'e478544eb8734c07a1943f5bdf4e6605be6943df', 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 0.5, "eta": 0.2, "gamma": 0, "max_depth": 7, "max_leaves": 7, "n_estimators": 25, "objective": "reg:logistic", "reg_alpha": 0, "reg_lambda": 0.20833333333333334, "subsample": 1, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "e478544eb8734c07a19

In [35]:
print(best_automl_run.get_properties())

{'runTemplate': 'automl_child', 'pipeline_id': 'e478544eb8734c07a1943f5bdf4e6605be6943df', 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 0.5, "eta": 0.2, "gamma": 0, "max_depth": 7, "max_leaves": 7, "n_estimators": 25, "objective": "reg:logistic", "reg_alpha": 0, "reg_lambda": 0.20833333333333334, "subsample": 1, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "e478544eb8734c07a1943f5bdf4e6605be6943df", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipeline_name": "{ StandardScaler, XGBoostClassifier }"}', 'training_percent': '100', 'predicted_cost': '3.3433926001021748', 'iteration': '21', '_aml_s

In [17]:
best_automl_run.register_model(model_name='AutoML_best_run', model_path='.')

Model(workspace=Workspace.create(name='quick-starts-ws-240238', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-240238'), name=AutoML_best_run, id=AutoML_best_run:1, version=1, tags={}, properties={})

Model deployment type 2

## Deploy model
TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [40]:
best_run = remote_run.get_best_child()
model_name = best_run.properties["model_name"]

script_file_name = "inference/score.py"

best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")


In [41]:
model = remote_run.register_model(
    model_name=model_name
)

In [42]:
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1
)

aci_service_name = model_name.lower()
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automlcc6d7769421
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-17 12:25:42+00:00 Creating Container Registry if not exists.
2023-08-17 12:25:43+00:00 Use the existing image.
2023-08-17 12:25:44+00:00 Submitting deployment to compute.
2023-08-17 12:25:49+00:00 Checking the status of deployment automlcc6d7769421..
2023-08-17 12:27:42+00:00 Checking the status of inference endpoint automlcc6d7769421.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Get the first data sample to test the endpoint on it

In [45]:
#get first sample from dataset to test endpoint
data_sample_pd = dataset.take(1).to_pandas_dataframe()
data_sample_pd.drop('diabetes',axis=1, inplace=True)

data_sample_json = data_sample_pd.to_json(orient="records")
data_sample = '{"data": ' + data_sample_json + "}"

Test the endpoint

In [48]:
import requests
import json

headers = {"Content-Type": "application/json"}

resp = requests.post(aci_service.scoring_uri, data_sample, headers=headers)

print(json.loads(json.loads(resp.text))["result"])

[0]


TODO: In the cell below, print the logs of the web service and delete the service

In [50]:
print(aci_service.get_logs())

2023-08-17T12:27:33,364366400+00:00 - rsyslog/run 
2023-08-17T12:27:33,374594900+00:00 - gunicorn/run 
2023-08-17T12:27:33,379694200+00:00 | gunicorn/run | 
2023-08-17T12:27:33,382301100+00:00 | gunicorn/run | ###############################################
2023-08-17T12:27:33,385450000+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-08-17T12:27:33,385573900+00:00 - nginx/run 
2023-08-17T12:27:33,387980500+00:00 | gunicorn/run | ###############################################
2023-08-17T12:27:33,391744200+00:00 | gunicorn/run | 
2023-08-17T12:27:33,405839000+00:00 | gunicorn/run | 
2023-08-17T12:27:33,416931800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2023-08-17T12:27:33,423828300+00:00 | gunicorn/run | 
2023-08-17T12:27:33,427538600+00:00 | gunicorn/run | 
2023-08-17T12:27:33,430855400+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [51]:
aci_service.delete()

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
